# Saco de palavras de 1-5 grams 

## Solução:
1. Criar uma lista de "stopwords" adequada para termos de "n" palavras;
2. Gerar uma lista de "tokens" removendo pontuação e as "stopwords";
3. Criar 5 grupos de ngrams, de 1 a 5 palavras;
4. Definir um limite de ocorrências mínimo;
5. Iterar entre os grupos, removendo os termos duplicados de ngrams menores, por exemplo:
    * Se houver um bigram (('sindicato', 'metalúrgico'), 5), os onegram (('sindicato',), 5) e (('metalúrgico',), 5) são removidos.
6. São somados os 5 grupos e retorna a lista de termos mais frequentes.

## Problemas:
1. As "stopwords" para onegram não são as mesmas para ngrams com duas ou mais palavras;
2. Como definir o limite de ocorrências?
3. Palavras que aparecem em um "ngram" de duas ou mais palavras são removidas das "ngrams" menores.

## Conclusões:
1. Quase não há perda de palavras quando usado apenas bigram e onegram.
2. A lista de "stopwords" deve ser dividida para cada grupo de ngrams.
3. O limite mínimo de ocorrências e a quantidade de discursos processados influenciam muito na qualidade do resultado.


In [447]:
from nltk import word_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords as nltk_stopwords
from collections import Counter
from string import punctuation
import requests
import re

# Endereço da API dos discursos
BABEL_API_URL = 'https://babel.labhackercd.leg.br/api/v1/manifestations?manifestation_type__id=1'

# Pegando os 200 primeiros discursos da API do Babel
speeches = []
data = requests.get(BABEL_API_URL).json()['results']
for i in range(2, 11):
    data += requests.get(BABEL_API_URL + '&page=%s' % i).json()['results']
    
for speech in data:
    for attr in speech['attrs']:
        if attr['field'] == 'original':
            speeches.append(attr['value'])
            break

In [471]:
# Criando a lista de stopwords em português
EXTRA_STOPWORDS = [
    'sr.', 'nesse', 'deputados', '``', "''", 'empresa', 'trabalhadores', 'tema', 'brasil', 'brasileiro',
    'brasileira', 'sociedade', 'grandes', 'meios', 'principal', 'deputada', 'nesta', 'valor', 'reais',
    'representante', 'brasileiros', 'necessidade', 'quero', 'ser', 'geral', 'todo', 'toda', 'estar',
    'ter', 'parlamentares', 'região', 'forma', 'parte', 'disso', 'debate', 'entregar', 'nessa', 'nome',
    'vez', 'quer', 'primeira', 'soberania', 'justamente', 'ponto', 'presentes', 'faz', 'futuro', 'podem',
    'maneira', 'falar', 'interesses', 'caso', 'espaço', 'entrega', 'deste', 'mesma', 'negócio', 'existe',
    'avançar', 'ano', 'diz', 'próprios', 'criação', 'própria', 'dizendo', 'trazer', 'preocupação', 'ali',
    'ficar', 'desse', 'importância', 'senhor', 'queremos', 'poderia', 'ir', 'próprio', 'área', 'segundo',
    'acontece', 'sei', 'sabem', 'comum', 'mim', 'tratar', 'obrigação', 'falando', 'discurso', 'muitas',
    'acabou', 'falou', 'outro', 'capacidade', 'força','querem', 'significa', 'serviço', 'dados', 'tentar',
    'ninguém', 'gente', 'ideia', 'deputadas', 'dentro', 'fazendo', 'mão', 'época', 'uso', 'fiz', 'último',
    'nenhum', 'números', 'alguma', 'acreditamos', 'achamos', 'pagar', 'paga', 'conjunto', 'contentam',
    'lacunas', 'esperamos', 'digo', 'sente', 'logo', 'pesquisar', 'principais', 'mudar', 'sequer', 'pagou',
    'pagando', 'traz', 'desafio', 'concreto', 'atende', 'tendo', 'aberto', 'curto', 'recebe', 'receber',
    'décadas', 'minutos', 'horas', 'segundos', 'motivo', 'posso', 'dando', 'entra', 'volto', 'construir',
    'algumas', 'passar', 'muita', 'nisso', 'deveria', 'dá', 'construir', 'muito', 'longo', 'muitas', 'outras',
    'existem', 'conseguimos', 'precisamos', 'feita', 'mencionou', 'falei', 'cujo', 'hora', 'dizia', 'venha',
    'conseguindo', 'conseguir', 'votar', 'alguém', 'somente', 'todas', 'fizemos', 'citar', 'saiba', 'boa',
    'deveriam', 'acontecendo', 'algo', 'apresentei', 'sabemos', 'outra', 'junto', 'daqui', 'aconteceu',
    'haver', 'sinto', 'preciso', 'muitos', 'minimamente', 'amanhã', 'ontem', 'partir', 'objetivo', 'opinião',
    'vivemos', 'discutindo', 'agradecemos', 'utilizados', 'contra', 'dona', 'falta', 'possa', 'manhã', 'novas',
    'após', 'pessoas', 'fundamental', 'desses', 'devido', 'item', 'século', 'domingo', 'sábado',
    'presidente', ',', '.', '...', 'é', 'questão', 'art', 'ordem', 'v.exa', ':', 'governo', 'sr', 'agência',
    'aqui', 'vai', 'artigo', '§', 'neste', 'vamos', 'agora', "''", 'fazer', 'mesa', 'ainda', 'porque', 'trata',
    'estrutura', 'sobre', 'então', 'todos', 'obstrução', 'votação', 'presença', 'deputados', 'vou', 'brasil',
    'discutir', 'vigência', 'colocar', 'regimento', 'momento', ';', 'dois', 'dessa', 'medida', 'proposta',
    'casa', 'matéria', 'queria', 'assim', 'possamos', 'microfone', 'certeza', 'hoje', 'profissional', 'deixar',
    'provisória', 'ora', 'base', 'importante', 'veto', 'fala', '!', 'aumento', 'inciso', 'sra.', 'talvez',
    'cima', 'servir', 'nunca', 'dias', 'deus', 'dei', 'entendemos', 'chega', 'possam', 'entendo', 'poderá',
    'celeridade', 'tirar', 'mista', 'fechou-se', 'lado', 'lido', 'repassado', 'demais', 'venho', 'marcar',
    'xiii', 'diálogo', 'podemos', 'apenas', 'poder', 'efeitos', 'pode', 'acordo', 'solicitação',
    'reflexão', '?', 'ausência', 'aprovada', 'lideranças', 'dizer', 'portanto', 'peço', 'recolher',
    'prática', 'pois', 'milhões', 'bilhões', 'melhoria', 'atividade', 'claro', 'saber', 'dar',
    'avanço', 'condições', 'desastre', 'especialmente', 'exatamente', 'política', 'vezes', 'fazê-lo', 'têm',
    'derrubar', 'precisa', 'custo', 'necessária', 'cláusula', 'proposição', '-', 'palavra', 'tempo', 'segundos',
    'fez', 'necessário', 'zero', 'interesse', 'srs', 'sr', 'sras', 'sra', 'deputado', 'presidente', 'é', 'nº',
    's.a.', 'v.exa.', 'v.exa', '#', 'anos', 'º', 'exa', 'mesa', 'legislatura', 'sessão', 'maioria', 'seguinte',
    'mandato', 'bilhões', 'quilômetros', 'ª', 'parabéns', 'membros', 'convido', 'usual', 'biênio',
    'brasil', 'palavra', 'discussão', 'período', 'início', 'pronunciamento', 'suplente', 'atividade', 'ação',
    'ações', 'daqueles', 'diferenças', 'pasta', 'milhares', 'srªs', 'emenda', 'àqueles', 'tamanha', 'mês',
    'capaz', 'km', 'modelo', 'tarefas', 'colegas', 'programa', 'voz', 'pronunciamento',
    'casa', 'sessão', 'deliberativa', 'solene', 'ordinária', 'extraordinária', 'encaminhado', 'orador',
    'divulgar', 'deputado', 'parlamentar', 'projeto', 'proposta', 'requerimento', 'destaque',
    'veto', 'câmara', 'senado', 'congresso', 'país', 'estado', 'brasil', 'lei', 'novo', 'nova',
    'política', 'povo', 'voto', 'partido', 'liderança', 'bancada', 'bloco', 'líder', 'lider', 'frente',
    'governo', 'oposição', 'presença', 'presente', 'passado', 'ausência', 'ausencia', 'ausente', 'obstrução',
    'registrar', 'aprovar', 'rejeitar', 'rejeição', 'sabe', 'matéria', 'materia', 'questão', 'ordem', 'emenda',
    'sistema', 'processo', 'legislativo', 'plenário', 'pedir', 'peço', 'comissão', 'especial', 'permanente',
    'apresentar', 'encaminhar', 'encaminho', 'orientar', 'liberar', 'apoiar', 'situação', 'fato', 'revisão',
    'tempo', 'pauta', 'discutir', 'discussão', 'debater', 'retirar', 'atender', 'colegas', 'autor', 'texto',
    'medida', 'união', 'república', 'audiência', 'audiencia', 'público', 'publico', 'reunião', 'agradecer',
    'solicitar', 'assistir', 'contrário', 'favorável', 'pessoa', 'comemorar', 'ato', 'momento', 'diretora',
    'possível', 'atenção', 'agradeço', 'naquele', 'necessárias', 'presidenta', 'compromisso', 'geradas',
    'primeiro', 'simplesmente', 'ideal', 'argumento', 'i', 'válido', 'envolvidos', 'nesse', 'aspecto',
    'existentes', 'normativo', 'irá', 'nada', 'melhor', 'esperarmos', 'pouco', 'resolvermos', 'problema',
    'postura', 'faltas', 'declara', '%', 'dia', 'obrigado', 'agradeço', 'agradecido', 'população',
    'maior', 'cada', 'bem', 'mundo', 'desta', 'mil', 'sendo', 'outros', '$', '!', '@', '#', '&', '(', ')',
    'r', 'sempre', 'além', 'semana', 'relação', 'onde', 'meio', 'inclusive', 'lá', 'vem', 'menos', 'menor',
    'qualquer', 'desde', 'ontem', 'hoje', 'exemplos', 'exemplo', 'tão', 'fim', 'janeiro', 'fevereiro', 'março',
    'abril', 'maio', 'junho', 'julho', 'agosto', 'setembro', 'outubro', 'novembro', 'dezembro', 'alguns',
    'durante', 'gostaria', 'três', 'conta', 'feito', 'através', 'antes', 'depois', 'verdade', 'bom', 'quase',
    'setor', 'aí', 'disse', 'principalmente', 'final', 'vão', 'coisa', 'ver', 'sentido', 'vários',
    'nenhuma', 'quanto', 'infelizmente', 'felizmente', 'número', 'duas', 'dois', 'tanto', 'acho', 'achar',
    'enquanto', 'deve', 'apelo', 'papel', 'últimos', 'faço', 'fazer', 'garantir', 'garantia', 'fica',
    'obrigado..', 'assunto', 'sido', 'vir', 'incrementar', 'central', 'aproximado', 'aproximadamente',
    'hipotética', 'hipotese', 'hipótese', 'superiores', 'entende', 'pedido', 'oradora', 'tal', 'v.exas',
    'favor', 'vota', 'nº', 'srª', 'vista', 'sim', 'dito', 'tudo', 'obrigado', 'º', 'profundamente', 'custódio',
    'divulgado', 'características', 'perfeito', 'começarmos', 'nomes', 'amigo', 'possibilidade', 'mensagem',
    'come', 'parabenizar', 'começar', 'hs', 'atendimento', 'povos', '¯', 'ocorreu', 'entanto', 'diante',
    'defender', 'dr.', '“', '”', '•', 'trabalhos', 'votaram', 'cumprimos', 'pautar', 'cargo', 'pudesse',
    'partidária', 'partidário', 'exercer', 'votada', 'suporte', 'comprometeu', 'deu', 'proporcional',
    'votando', 'dúvida', 'conhecimento', 'tipo', 'ficou', 'meses', 'lugar', 'destaques', 'encontro',
    'havia', 'várias', './', 'quais', 'dessas', 'passa', '°', 'devemos'
]


ONEGRAM_STOPWORDS = [
    'grande', 'nacional', 'são', 'e', 'de', 'das', 'dos', 'da', 'do', 'federal', 'cedo', 'urgência', 'equipe',
    'produtos', 'serviços', 'pequeno', 'total', 'podermos', 'consenso', 'popular', 'mérito', 'único', 'pública',
    'escolha', 'acesso', 'pilotos', 'trabalhar', 'ministério', 'países', 'combate', 'estados', 'vida', 'cidade',
    'municípios', 'histórico', 'defesa', 'município', 'prefeito', 'ii', 'santa', 'vereadora', 'centro',
    'prefeitura', 'governador', 'código', 'apoio', 'exercício', 'categoria', 'campo', 'kit', 'trabalho',
    'parlamento', 'candidato', 'partidos', 'vontade', 'legislação', 'mandatos', 'simples', 'aglutinativa',
    'cumprir', 'aguenta', 'direção', 'tribuna', 'serra', 'redução', 'campanha', 'empresas', 'fundo', 'capital',
    'crianças', 'real', 'campanhas', 'realmente', 'oportunidade',  'patrimônio', 'medidas', 'sociais',
    'plano', 'natural', 'áreas', 'obra', 'eleitoral', 'social', 'pesquisa', 'parque', 'candidatos', 'emendas',
    'participação', 'história', 'respeito', 'ministro', 'constituição', 'vagas', 'luta', 'dinheiro', 'registro',
    'comunicação', 'aprovação', 'casos', 'atual', 'trabalhador', 'projetos', 'públicos', 'coligações',
    'relatório', 'termos', 'cidades', 'santo', 'qualidade', 'constitucional', 'internacional', 'públicas',
    'tribunal', 'penal', 'relator', 'econômico', 'presidência', 'supremo', 'divulgação', 'aprovado', 'obras',
    'gestão', 'problemas', 'causa', 'direito', 'democracia', 'federais', 'órgãos', 'importantes', 'conquista',
    'índice', 'última', 'cidadãos', 'filho', 'grupo', 'leis', 'profissionais', 'novos', 'publicação', 'evento',
    'turno', 'planos', 'políticas', 'vereadores'
]


def get_tokens(speeches, extra_stopwords=None):
    """
    Função que retorna tokens de discursos removendo as "stopwords".
    Argumentos:
        speeches: Recebe uma lista de discursos.
        stopwords: Recebe uma lista de palavras a serem retiradas dos textos.
    Retorna:
        Uma lista palavras do discurso que não estão nas "stopwords".
    """
    special_stopwords = ['são']
    stopwords = nltk_stopwords.words('portuguese') + list(punctuation) + EXTRA_STOPWORDS
    stopwords = [word for word in stopwords if word not in special_stopwords]
    if extra_stopwords:
        stopwords += extra_stopwords
    tokens = []
    for text in speeches:
        text = re.sub(r'[0-9]+', '', text)
        tokens += [i for i in word_tokenize(text.lower(), language='portuguese') if i not in stopwords]
    
    return tokens


def ngrams_by_limit(tokens, n, limit):
    """
    Função que retorna a quantidade de amostras do gênero masculino e feminino.
    Argumentos:
        tokens: Recebe uma lista de tokens já processados pelo nltk.word_tokenize.
        n: Recebe o número de palavras que deseja dividir o ngram.
        limit: Recebe o limite mínimo de ocorrência.
    Retorna:
        Uma lista de ngrams com ocorrência maior que "limite" e com "n" palavras.
    """
    ngrams_count = Counter(ngrams(tokens, n)).most_common()
    result = [x[0] for x in ngrams_count if x[1] >= limit]
    return result

#Definimos os tokens
tokens = get_tokens(speeches)
onegram_tokens = get_tokens(speeches, ONEGRAM_STOPWORDS)

# Determinamos o limite de ocorrências usado no algoritmo
limit = Counter(onegram_tokens).most_common(100)[-1][1]
if limit < 2:
    limit = 2

# Agrupamos os ngrams em listas de strings
onegram = list(map(' '.join, ngrams_by_limit(onegram_tokens, 1, limit)))
bigrams = list(map(' '.join, ngrams_by_limit(tokens, 2, limit)))
trigrams = list(map(' '.join, ngrams_by_limit(tokens, 3, limit)))
fourgrams = list(map(' '.join, ngrams_by_limit(tokens, 4, limit)))
fivegrams = list(map(' '.join, ngrams_by_limit(tokens, 5, limit)))

print("Limite mínimo de ocorrências: %s" % limit)
print("Fivegrams[+5]: " + ", ".join(fivegrams[:5]))
print("Fourgrams[+5]: " + ", ".join(fourgrams[:5]))
print("Trigrams[+5]: " + ", ".join(trigrams[:5]))
print("Bigrams[+5]: " + ", ".join(bigrams[:5]))
print("Onegrans[+5]: " + ", ".join(onegram[:5]))

Limite mínimo de ocorrências: 19
Fivegrams[+5]: 
Fourgrams[+5]: 
Trigrams[+5]: redução maioridade penal, rio grande sul
Bigrams[+5]: maioridade penal, redução maioridade, rio grande, grande sul, são paulo
Onegrans[+5]: saúde, dilma, maioridade, educação, desenvolvimento


In [472]:
# Removemos os ngrams menores que já foram usados nos ngrams maiores
new_fourgrams = [b for b in fourgrams if all(b not in t for t in fivegrams)]
new_trigrams = [b for b in trigrams if all(b not in t for t in fourgrams + fivegrams)]
new_bigrams = [b for b in bigrams if all(b not in t for t in trigrams + fourgrams + fivegrams)]
new_onegram = [b for b in onegram if all(b not in t for t in bigrams + trigrams + fourgrams + fivegrams)]

# Resultado do saco de palavras com até 5 palavras
terms = fivegrams + new_fourgrams + new_trigrams + new_bigrams + new_onegram

print(", ".join(terms[:30]))

redução maioridade penal, rio grande sul, são paulo, segurança pública, eduardo cunha, judiciário federal, vitória conquista, arthur virgílio, saúde, dilma, educação, desenvolvimento, recursos, pt, agricultura, crise, crimes, justiça, servidores, direitos, menores, ruas, mulheres, jovens, josé, violência, economia, família, petrobras, bahia
